In [1]:
import pandas as pd
import pickle
import folium


In [2]:
# Load cleaned dataset
clean_df = pd.read_csv("../fc212025 udayanga/clean_data.csv")


In [3]:
# Create a 'date' column from year, month, and day if it doesn't exist
if 'date' not in clean_df.columns:
    clean_df['date'] = pd.to_datetime(clean_df[['year', 'month', 'day']])

# Get the latest available date (today)
today_date = clean_df['date'].max()
today_df = clean_df[clean_df['date'] == today_date].copy()

print(f"📅 Running prediction for: {today_date.strftime('%Y-%m-%d')}")


📅 Running prediction for: 2024-06-08


In [4]:
# Load the trained model
model_path = "../../models/random_forest.pkl"

with open(model_path, "rb") as f:
    model = pickle.load(f)


In [5]:
# Drop unused/non-numeric columns for prediction
exclude_cols = ['RainToday', 'date', 'city_name', 'country']
X_today = today_df.drop(columns=[col for col in exclude_cols if col in today_df.columns], errors='ignore')

# Select only numeric features
X_today = X_today.select_dtypes(include='number')


In [6]:
# Drop 'location_id' if present, to match training features
X_today_predict = X_today.drop(columns=['location_id'], errors='ignore')

# Predict rain (RainTomorrow = 1 means rain predicted)
today_df['RainPredicted'] = model.predict(X_today_predict)

# Filter for locations where rain is predicted
rain_predicted_df = today_df[today_df['RainPredicted'] == 1].copy()

print(f"🌧️ Predicted rain in {len(rain_predicted_df)} locations.")


🌧️ Predicted rain in 26 locations.


In [7]:
# Center of Sri Lanka for the map
map_center = [7.8731, 80.7718]

# Create interactive map
rain_map = folium.Map(location=map_center, zoom_start=7)

# Add markers for cities with predicted rain
for _, row in rain_predicted_df.iterrows():
    city = row.get("city_name", "Unknown")
    lat = row.get("latitude")
    lon = row.get("longitude")
    rain_mm = row.get("rain_sum (mm)", 0)

    # Add marker
    folium.Marker(
        location=[lat, lon],
        popup=f"{city}<br>Rain (mm): {rain_mm}",
        icon=folium.Icon(color="blue", icon="cloud")
    ).add_to(rain_map)

# Show map
rain_map


In [8]:
# Save the map to an HTML file
map_output_path = "../fc212025 udayanga/rain_prediction_map.html"
rain_map.save(map_output_path)

print(f"💾 Map saved to: {map_output_path}")


💾 Map saved to: ../fc212025 udayanga/rain_prediction_map.html
